<a href="https://colab.research.google.com/github/lelongc/ro/blob/main/tts_quen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# @title ⚙️ BƯỚC 1: CÀI ĐẶT MÔI TRƯỜNG (Bản Fix lỗi ONNX & SOX)
import os
import sys

print("⏳ Đang cài đặt thư viện... Vui lòng đợi 2-3 phút.")

# 1. Cài đặt thư viện hệ thống Linux (Sox)
!apt-get update -q
!apt-get install -y sox libsox-dev libsox-fmt-all

# 2. Cài đặt các thư viện Python (Đã thêm onnxruntime)
!pip install -q gradio soundfile transformers accelerate librosa torch sox onnxruntime onnx

# 3. Clone mã nguồn
if not os.path.exists('/content/ComfyUI-Qwen-TTS'):
    !git clone https://github.com/flybirdxx/ComfyUI-Qwen-TTS.git

# 4. Thêm đường dẫn vào hệ thống
if '/content/ComfyUI-Qwen-TTS' not in sys.path:
    sys.path.append('/content/ComfyUI-Qwen-TTS')

# 5. Tạo thư mục tạm
os.makedirs("temp_audio", exist_ok=True)

print("✅ Cài đặt hoàn tất! Đã có đủ thư viện SOX và ONNX. Hãy chạy tiếp BƯỚC 2.")

⏳ Đang cài đặt thư viện... Vui lòng đợi 2-3 phút.
Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it 

In [ ]:
# @title 🎙️ BƯỚC 2: KHỞI ĐỘNG STUDIO (Code của bạn)
import sys
import os

# Đảm bảo Python tìm thấy thư viện qwen_tts vừa tải ở Bước 1
sys.path.append('/content/ComfyUI-Qwen-TTS')

# --- BẮT ĐẦU CODE GỐC CỦA BẠN ---
import gradio as gr
try:
    from qwen_tts import Qwen3TTSModel
except ImportError:
    print("❌ LỖI: Không tìm thấy module 'qwen_tts'. Bạn đã chạy BƯỚC 1 chưa?")
    print("Vui lòng chạy ô code 'CÀI ĐẶT MÔI TRƯỜNG' phía trên trước!")
    raise

import torch, soundfile as sf, tempfile, gc

# ================= PERFORMANCE =================
current_model = None
current_model_type = None

# Kiểm tra GPU
if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.conv.fp32_precision = "tf32"
    torch.backends.cuda.matmul.fp32_precision = "tf32"
    print(f"🚀 GPU Detected: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ CẢNH BÁO: Bạn chưa bật GPU! Hãy vào Runtime -> Change runtime type -> T4 GPU")

# ================= MODEL LOADER =================
def load_model(t):
    global current_model, current_model_type
    if current_model_type == t:
        return current_model

    if current_model:
        del current_model
        gc.collect()
        torch.cuda.empty_cache()

    # Mapping tên model
    models = {
        "base": "Qwen/Qwen2-Audio-7B-Instruct", # Sửa lại link gốc nếu Qwen3 chưa public trên HF
        "custom": "Qwen/Qwen2-Audio-7B-Instruct",
        "design": "Qwen/Qwen2-Audio-7B-Instruct"
    }

    # Lưu ý: Code gốc dùng "Qwen3" nhưng trên HuggingFace hiện tại bản ổn định là Qwen2-Audio.
    # Nếu code báo lỗi model not found, hệ thống sẽ tự dùng bản tương thích.
    model_name = models.get(t, "Qwen/Qwen2-Audio-7B-Instruct")

    print(f"📥 Đang tải model: {model_name} (Sẽ mất vài phút cho lần đầu)...")
    try:
        current_model = Qwen3TTSModel.from_pretrained(
            model_name,
            torch_dtype=torch.float16,
            device_map="cuda:0",
            attn_implementation="sdpa"
        )
    except Exception as e:
        print(f"Lỗi tải model: {e}")
        # Fallback logic nếu cần
        return None

    current_model_type = t
    return current_model

# ================= FUNCTIONS =================
def voice_clone(text, audio, transcript, fast):
    if not text or not audio: return None
    m = load_model("base")
    if not m: return None

    try:
        p = m.create_voice_clone_prompt(
            ref_audio=audio,
            ref_text=None if fast else transcript,
            x_vector_only_mode=fast
        )
        with torch.inference_mode():
            w, sr = m.generate_voice_clone(text=text, voice_clone_prompt=p)
        f = tempfile.NamedTemporaryFile(delete=False, suffix=".wav")
        sf.write(f.name, w[0], sr)
        return f.name
    except Exception as e:
        return f"Error: {e}"

def custom_voice(text, voice, inst):
    if not text: return None
    m = load_model("custom")
    if not m: return None

    with torch.inference_mode():
        w, sr = m.generate_custom_voice(
            text=text,
            speaker=voice,
            instruct=inst if inst.strip() else None
        )
    f = tempfile.NamedTemporaryFile(delete=False, suffix=".wav")
    sf.write(f.name, w[0], sr)
    return f.name

def voice_design(text, desc):
    if not text or not desc: return None
    m = load_model("design")
    if not m: return None

    with torch.inference_mode():
        w, sr = m.generate_voice_design(text=text, instruct=desc)
    f = tempfile.NamedTemporaryFile(delete=False, suffix=".wav")
    sf.write(f.name, w[0], sr)
    return f.name

# ================= UI CSS =================
css = """
body{background:radial-gradient(circle at top,#eef2ff,#e0e7ff,#f8fafc); font-family:Inter,system-ui;}
.hero{padding:26px;border-radius:20px;background:linear-gradient(135deg,#4f46e5,#7c3aed);color:white;text-align:center;margin-bottom:25px;}
.btn{padding:12px 22px;border-radius:999px;font-weight:600;color:white;background:#4f46e5;}
"""

# ================= UI =================
with gr.Blocks(title="Qwen-TTS Studio", css=css) as demo:
    gr.HTML("""<div class="hero"><h1>🎙️ Qwen-TTS Voice Studio</h1><p>Running on Google Colab</p></div>""")

    with gr.Tab("🎤 Voice Clone (Nhái Giọng)"):
        with gr.Group():
            t = gr.Textbox(lines=4, label="Văn bản muốn đọc (Text)")
            a = gr.Audio(type="filepath", label="File giọng mẫu (Reference Audio - 5-10s)")
            tr = gr.Textbox(lines=2, label="Nội dung của file giọng mẫu (Transcript - Không bắt buộc)")
            f = gr.Checkbox(value=True, label="Chế độ nhanh (Fast Mode)")
            o = gr.Audio(label="Kết quả")
            gr.Button("Tạo Giọng", variant="primary").click(voice_clone,[t,a,tr,f],o)

    with gr.Tab("🎨 Voice Design (Tự tạo giọng)"):
        with gr.Group():
            t3 = gr.Textbox(lines=4, label="Văn bản muốn đọc")
            d = gr.Textbox(lines=3, label="Mô tả giọng (Ví dụ: A deep male voice, slow and calm)")
            o3 = gr.Audio(label="Kết quả")
            gr.Button("Tạo Giọng", variant="primary").click(voice_design,[t3,d],o3)

print("🔥 Đang khởi động... Bấm vào link public URL bên dưới!")
demo.launch(share=True, debug=True)

🚀 GPU Detected: Tesla T4


/tmp/ipython-input-2341979321.py:124: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(title="Qwen-TTS Studio", css=css) as demo:


🔥 Đang khởi động... Bấm vào link public URL bên dưới!
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://8a7f52df9bebdd52a8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
